In [ ]:
from gqimax.mapper import map_cx, map_noncx
from gqimax.sample import sample2, sample3
from gqimax.mapper import weightss_to_lambdas, map_indices_to_indicess, map_noncx
import cupy as cp
import time

import cupy as cp


num_qubits = 6
num_layers = 2
num_repeats = 1000
ins = sample3(num_qubits, num_layers, num_repeats)
ins.operatoring()
from gqimax.pstabilizers import PStabilizers
stb = PStabilizers(num_qubits)
lambdass = stb.lambdass
indicesss = stb.indicess

for j, order in enumerate(ins.orders):
    k = j // 2
    if order == 0:
        lambdass, indicesss = map_noncx(lambdass, indicesss, ins.lut[k])

    else:
        for _, indices, _ in ins.xoperators[k]:
            lambdass, indicesss = map_cx(lambdass, indicesss, indices[0], indices[1])

# print(lambdass[0])
# print(indicesss[0])

In [9]:
len(lambdass[0])

1260